In [1]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2021-02-07 13:53:13--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2021-02-07 13:53:13--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7792e3a658c661806a2ac30c26.dl.dropboxusercontent.com/cd/0/inline/BIeBMyzU847ZcEShIoJdBMxYuL2Q_6eBg14akmphFjKNPS2HcgsPzhzauPwoovMK5hliJdaZfUE8JP4K7rqt1l2Mp0pPyzyUbAnEKnDgS3jI4g/file# [following]
--2021-02-07 13:53:14--  https://uc7792e3a658c661806a2ac30c26.dl.dropboxusercontent.com/cd/0/inline/BIeBMyzU847ZcEShIoJdBMxYuL2Q_6eBg14akmphFjKNPS2HcgsPzhzauPwoovMK5hliJdaZfUE8JP4K7rqt1l2Mp0pPyzyUbAnEKnDgS3jI4g/file
Resolving uc7792

In [2]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv 

--2021-02-07 13:53:15--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2021-02-07 13:53:15--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc748f2ad7f816738b46efbb2ad3.dl.dropboxusercontent.com/cd/0/inline/BIdWc3Z6gICDGwKvLZNzK20fwi7ph-R7mQF-BHQwblrlsqhFBOdJiGrQisSvWLKTjAti8J98wN2k1mXlf_TGxMcodf-nhwpiUiLETb6Xje1pIA/file# [following]
--2021-02-07 13:53:16--  https://uc748f2ad7f816738b46efbb2ad3.dl.dropboxusercontent.com/cd/0/inline/BIdWc3Z6gICDGwKvLZNzK20fwi7ph-R7mQF-BHQwblrlsqhFBOdJiGrQisSvWLKTjAti8J98wN2k1mXlf_TGxMcodf-nhwpiUiLETb6Xje1pIA/file
Resolving uc748f

In [3]:
pip install gensim==3.7.3

     |████████████████████████████████| 24.2MB 1.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from gensim.models import KeyedVectors, FastText
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional, TimeDistributed, Conv1D, concatenate, SpatialDropout1D, Flatten
from tensorflow.keras.layers import Embedding, Input
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import Accuracy, Recall, Precision

In [3]:
colnames = ['id', 'date', 'name', 'text', 'type', 'rep', 'retweet', 'fav',  
            'stcount', 'foll', 'friends', 'listcount']

In [4]:
positive = pd.read_csv('positive.csv', sep=';', names=colnames)

In [5]:
negative = pd.read_csv('negative.csv', sep=';', names=colnames)

In [6]:
negative.head()

,id,date,name,text,type,rep,retweet,fav,stcount,foll,friends,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [7]:
positive.head()

,id,date,name,text,type,rep,retweet,fav,stcount,foll,friends,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [8]:
data = pd.concat([positive, negative])

In [9]:
data.head()

,id,date,name,text,type,rep,retweet,fav,stcount,foll,friends,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


## biLSTM model

Убираем все некириллические символы из датасета

In [10]:
sentences = []
types = []
for i, row in data.iterrows():
    sent = row['text'].lower()
    sent = re.sub('[^а-я]', ' ', sent)
    sentences.append(sent.split())
    t = row['type']
    types.append([t for i in range(len(sent.split()))])

In [11]:
sentences[0]

['хоть',
 'я',
 'и',
 'школота',
 'но',
 'поверь',
 'у',
 'нас',
 'то',
 'же',
 'самое',
 'общество',
 'профилирующий',
 'предмет',
 'типа']

In [12]:
types[0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [13]:
len(sentences[0]) == len(types[0])

True

In [14]:
sent_train, sent_test, type_train, type_test = train_test_split(sentences, types, test_size=0.2, random_state=1)

In [15]:
vocab = Counter()
for sent in sent_train:
    vocab.update(sent)

In [16]:
filtered_vocab = {word for word in vocab if vocab[word] > 5}

In [17]:
len(filtered_vocab)

23081

In [18]:
!wget http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2021-02-07 14:02:58--  http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz.2’

 araneum_none_fastt   1%[                    ]  29.72M  16.3MB/s               ^C


In [21]:
!tar --gunzip --extract --verbose --file=araneum_none_fasttextcbow_300_5_2018.tgz

araneum_none_fasttextcbow_300_5_2018.model
araneum_none_fasttextcbow_300_5_2018.model.vectors_ngrams.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors_vocab.npy


In [22]:
model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)

In [21]:
word2id = {word:i + 2 for i,word in enumerate(filtered_vocab)}
word2id['pad'] = 0
word2id['unk'] = 1

id2word = {i:word for word, i in word2id.items()}

In [24]:
weights = np.array([model[word] for word in id2word.values() if word in model])

In [25]:
len(id2word) == len(weights)

True

In [22]:
def data2ints(data, smth2id):
  int_data = []
  for seq in data:
      int_seq = []
      for i in seq:
        int_seq.append(smth2id.get(i, 1))
  
      int_data.append(int_seq)
  return int_data

In [23]:
type2id = {'pad':0, 'unk':1}  
for typ in type_train:
    for t in typ: 
        if t not in type2id:
            type2id[t] = len(type2id)

id2type = {i:t for t, i in type2id.items()}

In [24]:
id2type

{0: 'pad', 1: 'unk', 2: 1, 3: -1}

In [25]:
X_train_ids, X_test_ids = data2ints(sent_train, word2id), data2ints(sent_test, word2id)
y_train_ids, y_test_ids = data2ints(type_train, type2id), data2ints(type_test, type2id)


print(X_train_ids[0])
print(y_train_ids[0])

print(X_test_ids[0])
print(y_test_ids[0])

[14202, 13780, 21959, 10836, 6955, 11281, 11473, 14202, 3645, 9158, 19132, 12350, 19603, 10787, 1, 13729, 8665, 13159]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[19557, 18056, 21673, 13729, 15739, 1, 6816, 13729, 9282, 1]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [26]:
sent_max_len = max(len(x) for x in X_train_ids)
print("максимальная длина предложения:", sent_max_len)

максимальная длина предложения: 34


In [27]:
X_train, X_test = pad_sequences(X_train_ids, maxlen=sent_max_len, padding='post'), pad_sequences(X_test_ids, maxlen=sent_max_len, padding='post')
y_train, y_test = pad_sequences(y_train_ids, maxlen=sent_max_len, padding='post'), pad_sequences(y_test_ids, maxlen=sent_max_len, padding='post')

In [28]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(181467, 34) (181467, 34) (45367, 34) (45367, 34)


In [29]:
y_train, y_test = to_categorical(y_train, num_classes=len(type2id)), to_categorical(y_test, num_classes=len(type2id))

In [30]:
y_train.shape

(181467, 34, 4)

### Модель с готовыми эмбэддингами для русского языка

In [ ]:
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weights])(word_in)
lstm = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
out = TimeDistributed(Dense(len(type2id), activation="softmax"))(lstm)

model = Model(inputs= word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[Precision(), Recall(), Accuracy()])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 34)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 34, 300)           6924900   
_________________________________________________________________
bidirectional (Bidirectional (None, 34, 256)           439296    
_________________________________________________________________
time_distributed (TimeDistri (None, 34, 4)             1028      
Total params: 7,365,224
Trainable params: 440,324
Non-trainable params: 6,924,900
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=1024, epochs=3, verbose=1)

Epoch 1/3
177/177 [==============================] - 460s 3s/step - loss: 0.2073 - precision: 0.8807 - recall: 0.8805 - accuracy: 0.0558 - val_loss: 0.2087 - val_precision: 0.8782 - val_recall: 0.8781 - val_accuracy: 0.0374
Epoch 2/3
177/177 [==============================] - 458s 3s/step - loss: 0.2036 - precision: 0.8850 - recall: 0.8848 - accuracy: 0.0846 - val_loss: 0.2047 - val_precision: 0.8835 - val_recall: 0.8834 - val_accuracy: 0.1137
Epoch 3/3
177/177 [==============================] - 455s 3s/step - loss: 0.2005 - precision: 0.8880 - recall: 0.8878 - accuracy: 0.0963 - val_loss: 0.2023 - val_precision: 0.8858 - val_recall: 0.8856 - val_accuracy: 0.1184


### Модель с fasttext эмбэддингами

In [ ]:
ft_model = FastText(size=100)
ft_model.build_vocab(sentences=sentences)
ft_model.train(sentences=sentences, total_examples=len(sentences), epochs=5)

In [ ]:
weights = np.array([ft_model.wv[word] for word in id2word.values()])

In [ ]:
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, trainable=False, weights=[weights])(word_in)
lstm = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
out = TimeDistributed(Dense(len(type2id), activation="softmax"))(lstm)

model = Model(inputs= word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[Precision(), Recall(), Accuracy()])

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 34)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 34, 100)           2308300   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 34, 256)           234496    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 34, 4)             1028      
Total params: 2,543,824
Trainable params: 235,524
Non-trainable params: 2,308,300
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=1024, epochs=3, shuffle=True, verbose=1)

Epoch 1/3
178/178 [==============================] - 352s 2s/step - loss: 5.1069 - precision_2: 0.6721 - recall_2: 0.6541 - accuracy: 0.6947 - val_loss: 5.2391 - val_precision_2: 0.6750 - val_recall_2: 0.6750 - val_accuracy: 0.8364
Epoch 2/3
178/178 [==============================] - 346s 2s/step - loss: 5.2035 - precision_2: 0.6768 - recall_2: 0.6768 - accuracy: 0.8303 - val_loss: 2.5621 - val_precision_2: 0.8409 - val_recall_2: 0.8409 - val_accuracy: 0.5211
Epoch 3/3
178/178 [==============================] - 346s 2s/step - loss: 2.5901 - precision_2: 0.8392 - recall_2: 0.8392 - accuracy: 0.5254 - val_loss: 2.5610 - val_precision_2: 0.8411 - val_recall_2: 0.8411 - val_accuracy: 0.5086


Попробуем изменить параметры при инициализации fasttext модели



In [45]:
ft = FastText(sentences, size=100, iter=3)

In [46]:
weights = np.array([ft.wv[word] for word in id2word.values()])

In [51]:
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, trainable=False, weights=[weights])(word_in)
lstm = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
out = TimeDistributed(Dense(len(type2id), activation="softmax"))(lstm)

model = Model(inputs= word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[Precision(), Recall(), Accuracy()])

In [52]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 34)]              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 34, 100)           2308300   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 34, 256)           234496    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 34, 4)             1028      
Total params: 2,543,824
Trainable params: 235,524
Non-trainable params: 2,308,300
_________________________________________________________________


In [53]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=1024, epochs=3, shuffle=True, verbose=1)

Epoch 1/3
178/178 [==============================] - 313s 2s/step - loss: 5.1056 - precision_5: 0.6719 - recall_5: 0.6539 - accuracy: 0.6850 - val_loss: 5.2391 - val_precision_5: 0.6750 - val_recall_5: 0.6750 - val_accuracy: 0.8374
Epoch 2/3
178/178 [==============================] - 317s 2s/step - loss: 5.2290 - precision_5: 0.6756 - recall_5: 0.6756 - accuracy: 0.8377 - val_loss: 5.2391 - val_precision_5: 0.6750 - val_recall_5: 0.6750 - val_accuracy: 0.8374
Epoch 3/3
178/178 [==============================] - 314s 2s/step - loss: 5.2277 - precision_5: 0.6757 - recall_5: 0.6757 - accuracy: 0.8377 - val_loss: 5.2391 - val_precision_5: 0.6750 - val_recall_5: 0.6750 - val_accuracy: 0.8374


## biLSTM + CharCNN

In [54]:
chars = set([letter for word in filtered_vocab for letter in word])
n_chars = len(chars)
print(chars)

{'я', 'з', 'ч', 'и', 'с', 'х', 'л', 'г', 'й', 'а', 'к', 'м', 'ц', 'е', 'р', 'в', 'э', 'н', 'д', 'т', 'о', 'щ', 'б', 'п', 'ю', 'у', 'ж', 'ш', 'ф', 'ь', 'ъ', 'ы'}


In [55]:
char2id = {c: i + 2 for i, c in enumerate(chars)}
char2id["pad"] = 0
char2id["unk"] = 1

id2char = {i:char for char, i in char2id.items()}

In [56]:
char_max_len = max(len(x) for x in filtered_vocab)
print("максимальная длина слова:", char_max_len)

максимальная длина слова: 33


In [57]:
def make_X_char(sentences):
  X_char = []
  for sentence in sentences:
      sent_seq = []
      for i in range(sent_max_len):
          word_seq = []
          for j in range(char_max_len):
              try:
                  word_seq.append(char2id[sentence[i][j].lower()])
              except:
                  word_seq.append(char2id["pad"])
          sent_seq.append(word_seq)
      X_char.append(np.array(sent_seq))
  return np.array(X_char)

In [58]:
X_char_train, X_char_test = make_X_char(sent_train), make_X_char(sent_test)

In [59]:
X_char_train.shape

(181467, 34, 33)

In [60]:
sent_train[5][0]

'хахахахаха'

In [61]:
X_char_train[5][0]

array([ 7, 11,  7, 11,  7, 11,  7, 11,  7, 11,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [63]:
weights = np.array([ft.wv[word] for word in id2word.values()])

Так как в моделях LSTM были низкие или нестабильные показатели качества, попробуем сейчас избавиться от переобучения изменением параметра dropout

In [76]:
# один вход для слов
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, trainable=False, weights=[weights])(word_in)

# другой вход для символов
char_in = Input(shape=(sent_max_len, char_max_len))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_max_len))(char_in)
# свертка на символах (CharRNN) применяется к каждому слову отдельно
char_enc = TimeDistributed(Conv1D(filters=12, kernel_size=3))(emb_char)
char_flat = TimeDistributed(Flatten())(char_enc)

# LSTM проходится по всей последовательности, на каждом шаге беря эмбеддинг слова по словарю + символьный эмбеддинг слова 
x = concatenate([emb_word, char_flat])
main_lstm = Bidirectional(LSTM(units=128, return_sequences=True,
                               recurrent_dropout=0.5))(x)
out = TimeDistributed(Dense(len(type2id), activation="softmax"))(main_lstm)

model = Model(inputs=[char_in, word_in], outputs=out)


optimizer = optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[Precision(), Recall(), Accuracy()])

In [77]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 34, 33)]     0                                            
__________________________________________________________________________________________________
time_distributed_22 (TimeDistri (None, 34, 33, 10)   340         input_18[0][0]                   
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
time_distributed_23 (TimeDistri (None, 34, 31, 12)   372         time_distributed_22[0][0]        
___________________________________________________________________________________________

In [78]:
model.fit([X_char_train, X_train], y_train, validation_data=([X_char_test, X_test], y_test), batch_size=1024, epochs=3, verbose=1)

Epoch 1/3
178/178 [==============================] - 744s 4s/step - loss: 5.1083 - precision_9: 0.6715 - recall_9: 0.6535 - accuracy: 0.6658 - val_loss: 5.2391 - val_precision_9: 0.6750 - val_recall_9: 0.6750 - val_accuracy: 0.8375
Epoch 2/3
178/178 [==============================] - 735s 4s/step - loss: 5.2275 - precision_9: 0.6757 - recall_9: 0.6757 - accuracy: 0.8378 - val_loss: 5.2391 - val_precision_9: 0.6750 - val_recall_9: 0.6750 - val_accuracy: 0.8375
Epoch 3/3
178/178 [==============================] - 720s 4s/step - loss: 5.2374 - precision_9: 0.6751 - recall_9: 0.6751 - accuracy: 0.8375 - val_loss: 5.2391 - val_precision_9: 0.6750 - val_recall_9: 0.6750 - val_accuracy: 0.8375


## biLSTM + CharCNN с другим отфильтрованным списком слов

In [99]:
filtered_vocab = {word for word in vocab if vocab[word] > 50}

In [100]:
len(filtered_vocab)

3431

In [101]:
word2id = {word:i + 2 for i,word in enumerate(filtered_vocab)}
word2id['pad'] = 0
word2id['unk'] = 1

id2word = {i:word for word, i in word2id.items()}

In [102]:
X_train_ids, X_test_ids = data2ints(sent_train, word2id), data2ints(sent_test, word2id)
y_train_ids, y_test_ids = data2ints(type_train, type2id), data2ints(type_test, type2id)


print(X_train_ids[0])
print(y_train_ids[0])

print(X_test_ids[0])
print(y_test_ids[0])

[3064, 2068, 1, 1999, 2999, 2935, 2, 3064, 2145, 1339, 1069, 2082, 2173, 1907, 1, 1941, 1, 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2053, 1, 128, 1941, 3324, 1, 1, 1941, 1, 1]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [103]:
sent_max_len = max(len(x) for x in X_train_ids)
print("максимальная длина предложения:", sent_max_len)

максимальная длина предложения: 34


In [104]:
X_train, X_test = pad_sequences(X_train_ids, maxlen=sent_max_len, padding='post'), pad_sequences(X_test_ids, maxlen=sent_max_len, padding='post')
y_train, y_test = pad_sequences(y_train_ids, maxlen=sent_max_len, padding='post'), pad_sequences(y_test_ids, maxlen=sent_max_len, padding='post')

In [105]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(181467, 34) (181467, 34) (45367, 34) (45367, 34)


In [106]:
y_train, y_test = to_categorical(y_train, num_classes=len(type2id)), to_categorical(y_test, num_classes=len(type2id))

In [107]:
chars = set([letter for word in filtered_vocab for letter in word])
n_chars = len(chars)
print(chars)

{'я', 'з', 'ч', 'и', 'с', 'х', 'л', 'г', 'й', 'а', 'к', 'м', 'ц', 'е', 'р', 'в', 'э', 'т', 'н', 'д', 'о', 'щ', 'б', 'п', 'ю', 'у', 'ш', 'ж', 'ф', 'ь', 'ъ', 'ы'}


In [108]:
char2id = {c: i + 2 for i, c in enumerate(chars)}
char2id["pad"] = 0
char2id["unk"] = 1

id2char = {i:char for char, i in char2id.items()}

In [109]:
char_max_len = max(len(x) for x in filtered_vocab)
print("максимальная длина слова:", char_max_len)

максимальная длина слова: 33


In [110]:
X_char_train, X_char_test = make_X_char(sent_train), make_X_char(sent_test)

In [112]:
weights = np.array([ft.wv[word] for word in id2word.values()])

Изменим также параметр learning rate

In [116]:
# один вход для слов
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, trainable=False, weights=[weights])(word_in)

# другой вход для символов
char_in = Input(shape=(sent_max_len, char_max_len))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_max_len))(char_in)
# свертка на символах (CharRNN) применяется к каждому слову отдельно
char_enc = TimeDistributed(Conv1D(filters=12, kernel_size=3))(emb_char)
char_flat = TimeDistributed(Flatten())(char_enc)

# LSTM проходится по всей последовательности, на каждом шаге беря эмбеддинг слова по словарю + символьный эмбеддинг слова 
x = concatenate([emb_word, char_flat])
main_lstm = Bidirectional(LSTM(units=128, return_sequences=True,
                               recurrent_dropout=0.5))(x)
out = TimeDistributed(Dense(len(type2id), activation="softmax"))(main_lstm)

model = Model(inputs=[char_in, word_in], outputs=out)


optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[Precision(), Recall(), Accuracy()])

In [117]:
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           [(None, 34, 33)]     0                                            
__________________________________________________________________________________________________
time_distributed_42 (TimeDistri (None, 34, 33, 10)   340         input_29[0][0]                   
__________________________________________________________________________________________________
input_28 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
time_distributed_43 (TimeDistri (None, 34, 31, 12)   372         time_distributed_42[0][0]        
___________________________________________________________________________________________

In [118]:
model.fit([X_char_train, X_train], y_train, validation_data=([X_char_test, X_test], y_test), batch_size=1024, epochs=3, verbose=1)

Epoch 1/3
178/178 [==============================] - 725s 4s/step - loss: 0.3757 - precision_14: 0.8509 - recall_14: 0.7496 - accuracy: 0.0000e+00 - val_loss: 0.2226 - val_precision_14: 0.8618 - val_recall_14: 0.8600 - val_accuracy: 0.0000e+00
Epoch 2/3
178/178 [==============================] - 720s 4s/step - loss: 0.2188 - precision_14: 0.8673 - recall_14: 0.8655 - accuracy: 0.0000e+00 - val_loss: 0.2144 - val_precision_14: 0.8742 - val_recall_14: 0.8736 - val_accuracy: 0.0000e+00
Epoch 3/3
178/178 [==============================] - 721s 4s/step - loss: 0.2096 - precision_14: 0.8797 - recall_14: 0.8790 - accuracy: 0.0000e+00 - val_loss: 0.2060 - val_precision_14: 0.8829 - val_recall_14: 0.8824 - val_accuracy: 0.0000e+00


Accuracy очень маленькое, оставим только эту метрику и посмотрим, изменится ли результат

In [119]:
# один вход для слов
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, trainable=False, weights=[weights])(word_in)

# другой вход для символов
char_in = Input(shape=(sent_max_len, char_max_len))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_max_len))(char_in)
# свертка на символах (CharRNN) применяется к каждому слову отдельно
char_enc = TimeDistributed(Conv1D(filters=12, kernel_size=3))(emb_char)
char_flat = TimeDistributed(Flatten())(char_enc)

# LSTM проходится по всей последовательности, на каждом шаге беря эмбеддинг слова по словарю + символьный эмбеддинг слова 
x = concatenate([emb_word, char_flat])
main_lstm = Bidirectional(LSTM(units=128, return_sequences=True,
                               recurrent_dropout=0.5))(x)
out = TimeDistributed(Dense(len(type2id), activation="softmax"))(main_lstm)

model = Model(inputs=[char_in, word_in], outputs=out)


optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])

In [120]:
model.fit([X_char_train, X_train], y_train, validation_data=([X_char_test, X_test], y_test), batch_size=1024, epochs=3, verbose=1)

Epoch 1/3
178/178 [==============================] - 724s 4s/step - loss: 0.3655 - acc: 0.8023 - val_loss: 0.2212 - val_acc: 0.8639
Epoch 2/3
178/178 [==============================] - 720s 4s/step - loss: 0.2182 - acc: 0.8678 - val_loss: 0.2119 - val_acc: 0.8770
Epoch 3/3
178/178 [==============================] - 719s 4s/step - loss: 0.2100 - acc: 0.8787 - val_loss: 0.2060 - val_acc: 0.8827


В заключение, все модели показывают результаты, которые достаточно трудно сравнивать между собой. Например, для первой модели lstm с векторами fastext длиной 100 (пробовались также другие  длины векторов, что не привело к улучшению результата) recall, precision и accuracy падают, для второй модели lstm с векторами fastext длиной 100 с каждой эпохой остаются примерно одинаковыми. Единственной моделью lstm, которая при обучении
показывала стабильное улучшение результатов, была lstm с готовыми эмбэддингами для русского языка, но она не была выбрана для lstm + charcnn, потому что после третьей эпохи показатель accuracy оставался низким (возможно, при большем количестве эпох модель обучилась бы лучше, но обучение занимало достаточно продолжительное время). Несмотря на это, добавление charcnn улучшило работу модели как в смысле прогресса обучения, так и в более высоких всех показателей качества. 